In [1]:
from __future__ import print_function, division
import sys
sys.path.insert(0, 'lib')
import numpy as np
import random
import pydicom
import os
import matplotlib.pyplot as plt
import pickle
import math
import pydicom
from shutil import copyfile
import nibabel as nib
import scipy.ndimage as ndimage
from scipy.stats import pearsonr, spearmanr

from utils import make_giant_mat, make_dictionary, make_echo_dict
from difference_map_utils import make_difference
from cluster_utils import threshold_diffmaps, strip_empty_lines, resize
from lesion_utils import *
from inference_utils import run_inference
from make_inference_csv import *
from compare_segmentations import get_dice_scores, get_jaccard_indices, compare_segmentation_masks, compare_region_means, compare_region_changes
from loss_functions import coefficient_of_variation
from figure_utils import plot_mean_val_comparisons


Using TensorFlow backend.


In [5]:
import zipfile
from calculate_t2 import fit_t2
from segmentation_refinement import *
from projection import *
from inference_utils import *
import time
import shutil
from skimage.transform import resize

In [3]:
def get_slices(scan_dir):
    '''
    scan_dir = path to folder containing dicoms such that each dicom represents one slice of the image volume
    '''
    list_of_slices = glob.glob("{}/**".format(scan_dir),
                               recursive=True)
    return list(filter(is_dicom, list_of_slices))

def is_dicom(fullpath):
    if os.path.isdir(fullpath):
        return False
    _, path = os.path.split(fullpath)
    
    path = path.lower()
    if path[-4:] == ".dcm":
        return True
    if not "." in path:
        return True
    return False


In [ ]:
input_dir = '/home/ubuntu/sample_longitudinal_data' #'./input'#
vol_zip_list = np.sort([os.path.join(input_dir,i) for i in os.listdir(input_dir) if i[-4:]=='.zip'])

# Get model
model_weight_file = './model_weights/model_weights_quartileNormalization_echoAug.h5'
model = get_model(model_weight_file)

# Prepare CSV to write all results to
region_list = ['all', 'superficial', 'deep','L', 'M', 'LA', 'LC', 'LP', 'MA', 'MC', 'MP', 'SL', 'DL', 'SM', 'DM','SLA', 'SLC', 'SLP', 'SMA', 'SMC', 'SMP', 'DLA', 'DLC', 'DLP', 'DMA', 'DMC', 'DMP']
output_file = open(os.path.join(input_dir,'predictions.csv'), 'w')
output_file.write('filename,'+','.join(region_list)+'\n')

total_time = 0
for zip_num, vol_zip in enumerate(vol_zip_list):
    print("Processing file %s..." % os.path.basename(vol_zip))
    time1 = time.time()
    new_dir_name = os.path.join('./output', os.path.splitext(os.path.basename(vol_zip))[0])
    os.makedirs(new_dir_name, exist_ok=True)
    dicom_sub_dir = os.path.join(new_dir_name,"dicom")
    raw_extract_dir = os.path.join(new_dir_name,"raw_extract")
    os.makedirs(dicom_sub_dir, exist_ok=True)
    
    # Unzip the image volume. If the zip file contains an inner directory, move the files out of it. 
    with zipfile.ZipFile(vol_zip, 'r') as zip_ref:
        zip_ref.extractall(raw_extract_dir)
    
    slice_path_list = get_slices(raw_extract_dir)
    for s in slice_path_list:
        shutil.copy(s,os.path.join(dicom_sub_dir, os.path.basename(s)))
    shutil.rmtree(raw_extract_dir)
    

    mese, times = assemble_4d_mese_v2(dicom_sub_dir)
    
    # If the slices are not 384x384, resize them
    mese_resized = np.zeros((mese.shape[0], mese.shape[1], 384,384))
    for s in range(mese.shape[0]):
        for echo in range(mese.shape[1]):
            mese_resized[s,echo,:,:] = resize(mese[s,echo,:,:], (384, 384),anti_aliasing=True)
    mese = mese_resized
    
    
    # Whiten the echo of each slice that is closest to 10ms 
    mese_white = []
    for i,s in enumerate(mese):
        if times[i][0] is not None:
            slice_times = times[i]
        else:
            times[i][0]=times[i][1]-(times[i][2]-times[i][1])
            slice_times = times[i]
        slice_20ms_idx = np.argmin(slice_times-.02)
        mese_white.append(whiten_img(s[slice_20ms_idx,:,:], normalization = 'quartile'))
    mese_white = np.stack(mese_white).squeeze()
    
    
    
    # Estimate segmentation
    seg_pred = model.predict(mese_white.reshape(-1,384,384,1), batch_size = 6)
    seg_pred = seg_pred.squeeze()
    
    # Calculate T2 Map
    t2 = fit_t2(mese, times, segmentation = seg_pred, n_jobs = 4, show_bad_pixels = False)
    
    # Refine the comparison segmentation by throwing out non-physiologic T2 values
    print("A:", np.sum(seg_pred))
    seg_pred, t2 = t2_threshold(seg_pred, t2, t2_low=0, t2_high=100)
    print("B:", np.sum(seg_pred))
    seg_pred, t2 = optimal_binarize(seg_pred, t2, prob_threshold=0.501, voxel_count_threshold=425)
    print("C:", np.sum(seg_pred))
    
    angular_bin = 5
    visualization, thickness_map, min_rho_map, max_rho_map, avg_vals_dict, R = projection(t2, 
                                                                                       thickness_div = 0.5, 
                                                                                       values_threshold = 100,
                                                                                       angular_bin = angular_bin, 
                                                                                       region_stat = 'mean',
                                                                                       fig = False)

    row_distance, column_distance = get_physical_dimensions(img_dir = dicom_sub_dir, 
                                                            t2_projection = visualization, 
                                                            projection_pixel_radius = R, 
                                                            angular_bin = angular_bin)
    
    t2_projection_dict = {}
    t2_projection_dict['t2_projection'] = visualization
    t2_projection_dict['thickness_map'] = thickness_map
    t2_projection_dict['row_distance'] = row_distance
    t2_projection_dict['column_distance'] = column_distance

    # Save the t2 image, segmentation, and projection results
    
    ## Save the 3D binary segmentation mask as a numpy array
    refined_seg_path = os.path.join(new_dir_name,"segmentation_mask.npy")
    np.save(refined_seg_path, seg_pred)
    
    ## Save the 3D binary segmentation mask as a folder of CSV files
    seg_sub_dir = os.path.join(new_dir_name,"segmentation_mask_csv")
    os.makedirs(seg_sub_dir, exist_ok=True)
    
    for i,s in enumerate(seg_pred):
        slice_path = os.path.join(seg_sub_dir,str(i).zfill(3)+".csv")
        np.savetxt(slice_path, s,delimiter=",", fmt='%10.5f')
                
    ## Save the 3D T2 image as a numpy array
    t2_img_path = os.path.join(new_dir_name,"t2.npy")
    np.save(t2_img_path, t2)
    
    ## Save the 3D T2 image as a folder of CSV files
    t2_sub_dir = os.path.join(new_dir_name,"t2_csv")
    os.makedirs(t2_sub_dir, exist_ok=True)
    
    for i,s in enumerate(t2):
        slice_path = os.path.join(t2_sub_dir,str(i).zfill(3)+".csv")
        np.savetxt(slice_path, s,delimiter=",", fmt='%10.5f')
    
    ## Save the 2D projection of the T2 map as a numpy array
    t2_projection_path = os.path.join(new_dir_name,"t2_projection.npy")
    np.save(t2_projection_path, visualization)
    
    ## Save the 2D projection of the T2 map as a csv
    t2_projection_csv_path = os.path.join(new_dir_name,"t2_projection.csv")
    np.savetxt(t2_projection_csv_path, visualization,delimiter=",", fmt='%10.5f')
    
    ## Save the 2D projection thickness map as a numpy array
    thickness_projection_path = os.path.join(new_dir_name,"thickness_projection.npy")
    np.save(thickness_projection_path, thickness_map)
    
    ## Save the 2D projection thickness map as a csv
    thickness_projection_csv_path = os.path.join(new_dir_name,"thickness_projection.csv")
    np.savetxt(thickness_projection_csv_path, thickness_map,delimiter=",", fmt='%10.5f')
    
    ## Save the physical dimensions of the 2D projections as a json
    projection_dimensions_dict = {}
    projection_dimensions_dict['row_distance(mm)'] = row_distance
    projection_dimensions_dict['column_distance(mm)'] = column_distance
    projection_dimensions_dict_path = os.path.join(new_dir_name,"projection_dimensions.json")
    with open(projection_dimensions_dict_path, 'w') as fp:
        json.dump(projection_dimensions_dict, fp)
        
    ## Save the region average T2 dictionary as a json
    t2_region_json_path = os.path.join(new_dir_name,"region_mean_t2.json")
    with open(t2_region_json_path, 'w') as fp:
        json.dump(avg_vals_dict, fp)
    
    time2 = time.time()
    total_time = total_time + (time2-time1)
    avg_pace = total_time / (zip_num+1)
    files_remaining = len(vol_zip_list) - zip_num
    print("Estimated time remaining (min):",np.round(files_remaining*avg_pace/60,decimals=1))    
    
    output_file.write('%s,' % os.path.basename(vol_zip))
    for r in region_list:
        if r == 'DMP':
            output_file.write('%d' % avg_vals_dict[r])
        else:
            output_file.write('%d,' % avg_vals_dict[r])

    output_file.write('\n')    

output_file.close()
    
    

In [ ]:
mask = np.load('/home/ubuntu/AutomaticKneeMRISegmentation/output/Hobart/t2_projection.npy')
plt.imshow(mask)

In [ ]:
region_list = ['all', 'superficial', 'deep','L', 'M', 'LA', 'LC', 'LP', 'MA', 'MC', 'MP', 'SL', 'DL', 'SM', 'DM','SLA', 'SLC', 'SLP', 'SMA', 'SMC', 'SMP', 'DLA', 'DLC', 'DLP', 'DMA', 'DMC', 'DMP']

','.join(region_list)

In [ ]:
plt.imshow(visualization)
plt.show()
print(visualization.shape)
print(row_distance)
print(column_distance)
avg_vals_dict

# Generate Segmentations and T2 maps for your MESE images
- If you provide a value for the 'expert_pd' argument, it will use your provided segmentations

- If you provide a value for the 'to_segment_pd' argument, it will automatically segment the cartilage and then use that auto-segmentation to generate the T2 maps. By default, this uses our trained model, but model weights can be changed via the 'model_weights_file' argument and the model can be changed by altering the inference.get_model function in inference.py. 

- In addition to generating 3D T2 maps, it also provides the segmentations used to generate those T2 maps as 3D numpy arrays and json files that summarize the avg T2 value in each anatomical region of the cartilage plate

- These results are all saved in the destinations specied in your Pandas dataframe (expert_pd or to_segment_pd) that you made in the previous step

In [ ]:
run_inference(to_segment_pd = predict_pd, model_weight_file = './model_weights/model_weights_quartileNormalization_echoAug.h5')
# run_inference(expert_pd = predict_pd)

# run_inference(expert_pd = expert1_pd)
              
# run_inference(expert_pd = expert2_pd)

# We don't need to generate additional segmentations for the 'predict_subset_pd' or 'expert1_subset_pd' 
# because they are already generated as part of the 'predict_pd' and 'expert1_pd'

In [ ]:
# Flip the expert segmentations if you haven't already
# for file in os.listdir('/data/kevin_data/expert2/segmentations'):
#     if file[-4:]=='.npy':
#         temp = np.load(os.path.join('/data/kevin_data/expert2/segmentations',file))
#         temp = np.flip(temp, axis = 0)
#         np.save(os.path.join('/data/kevin_data/expert2/segmentations',file),temp)

# Use cluster analysis to identify cartilage lesions developing over time
<img src="ClusterAnalysisExample.png" alt="Drawing" style="width: 200px;"/>

Find the percentage of the femoral cartilage surface area that is affected by cartilage lesions. In this context, a lesion is a localized area of cartilage that has increased in T2 value over time more than the surrounding area. You can adjust the settings in calculate_group_lesion_area() to make the criteria for lesions more or less strict based on how large a cluster must be and how much the T2 value must increase. 

Lesions are identified using methods adapted from the following manuscript:

Monu, Uchechukwuka D., et al. "Cluster analysis of quantitative MRI T2 and T1ρ relaxation times of cartilage identifies differences between healthy and ACL-injured individuals at 3T." Osteoarthritis and cartilage 25.4 (2017): 513-520.



In [ ]:
paths = np.array(predict_pd.t2_projected_path)
paths = [i + '.pickle' for i in paths]
source1_time1 = np.sort([i for i in paths if i[-12]=='4'])
source1_time2 = np.sort([i for i in paths if i[-12]=='8'])

paths = np.array(expert1_pd.t2_projected_path)
paths = [i + '.pickle' for i in paths]
source2_time1 = np.sort([i for i in paths if i[-12]=='4'])
source2_time2 = np.sort([i for i in paths if i[-12]=='8'])

percentLesion_expert1 = calculate_group_lesion_area(timepoint1=source2_time1,
                                                     timepoint2=source2_time2, 
#                                                      value_threshold = 9,
                                                     sigma_multiple = 1,
#                                                      area_value_threshold = 12.4,
                                                     area_fraction_threshold = .01, 
                                                     area_percentile_threshold = None,
                                                     display=False,
                                                     save_path_list=None)

print()
print("--"*10)


percentLesion_predict = calculate_group_lesion_area(timepoint1=source1_time1,
                                                     timepoint2=source1_time2, 
#                                                      value_threshold = 9,
                                                     sigma_multiple = 1,
#                                                      area_value_threshold = 12.4,
                                                     area_fraction_threshold = .01, 
                                                     area_percentile_threshold = None,
                                                     display=False,
                                                     save_path_list = None)

    

# SAVE THE RESIZED PROJECTIONS AND LESION MAPS!




# Compare two segmentation approaches (e.g. manual vs automated)
- Quantify how closely the two segmentations agree with one another using Dice Score and Jaccard Index
- Quantify how closely the downstream T2 measurements correlate for each region using Pearson correlation
- Quantify the mean absolute difference in T2 measurements for each region
- Quantify the agreement in the percentage of the cartilage plate that has lesion via Pearson correlation
- Quantify the agreement in the lesions identified using Dice Score and Jaccard Index

### Quantify how closely the two segmentations agree with one another using Dice Score and Jaccard Index


In [ ]:
print("Qmetric vs Model: Full test set")
dice_expert1_model, jaccard_expert1_model = compare_segmentation_masks(expert1_pd, 
                                                                       predict_pd, 
                                                                       display = True)

print("-"*100)
print()
print("Qmetric vs Model: Test subset")
dice_expert1_model_subset, jaccard_expert1_model_subset = compare_segmentation_masks(expert1_subset_pd, 
                                                                                     predict_subset_pd, 
                                                                                     display = True)

print("-"*100)
print("Qmetric vs Expert 2: Test subset")

dice_expert1_expert2, jaccard_expert1_expert2 = compare_segmentation_masks(expert1_subset_pd, 
                                                                           expert2_pd, 
                                                                           display = True)

print("-"*100)
print("Model vs Expert 2: Test subset")

dice_expert1_expert2, jaccard_expert1_expert2 = compare_segmentation_masks(predict_subset_pd, 
                                                                           expert2_pd, 
                                                                           display = True)


### See how well they agree in terms of the average T2 value in each cartilage region of each image

In [ ]:
correlation_dict_e1_m, abs_diff_dict_e1_m, cv_e1_m, t_e1_m, cohen_e1_m = compare_region_means(list(expert1_pd.t2_region_json_path), 
                                                                            list(predict_pd.t2_region_json_path), 
                                                                            results_path=None,
                                                                            correlation_method = 'spearman',
                                                                            bland_altman = False)

correlation_dict_e1_m_subset, abs_diff_dict_e1_m_subset, cv_e1_m_subset, t_e1_m_subset, cohen_e1_m_subset = compare_region_means(list(expert1_subset_pd.t2_region_json_path), 
                                                                                                list(predict_subset_pd.t2_region_json_path), 
                                                                                                results_path=None,
                                                                                                correlation_method = 'spearman')

correlation_dict_e1_e2, abs_diff_dict_e1_e2, cv_e1_e2, t_e1_e2, cohen_e1_e2 = compare_region_means(list(expert1_subset_pd.t2_region_json_path), 
                                                                            list(expert2_pd.t2_region_json_path), 
                                                                            results_path=None,
                                                                            correlation_method = 'spearman')

correlation_dict_e2_m, abs_diff_dict_e2_m, cv_e2_m, t_e2_m, _cohen_e1_e2 = compare_region_means(list(expert2_pd.t2_region_json_path), 
                                                                            list(predict_subset_pd.t2_region_json_path), 
                                                                            results_path=None,
                                                                            correlation_method = 'spearman')

plot_mean_val_comparisons(abs_diff_dict_e1_e2, 
                          abs_diff_dict_e1_m_subset, 
                          'Reader 2', 
                          'Model',
                          error_bar='ci')

# plot_mean_val_comparisons(abs_diff_dict_e1_e2, 
#                           abs_diff_dict_e1_m_subset, 
#                           'Reader 2', 
#                           'Model',
#                           error_bar='ci')

# plot_mean_val_comparisons(abs_diff_dict_e1_e2, 
#                           abs_diff_dict_e2_m, 
#                           'Reader 1', 
#                           'Model',
#                           error_bar = 'ci')


roi_list = np.array(['SLA', 'SLC', 'SLP', 'SMA', 'SMC', 'SMP', 'DLA', 'DLC', 'DLP', 'DMA', 'DMC', 'DMP'])
print("CORRELATION: EXPERT1 vs Model (SUBSET)")
roi_corr_mean = 0
for k,v in correlation_dict_e1_m_subset.items():
    print(k, v)
    if k in roi_list:
        roi_corr_mean += v[0]
roi_corr_mean = roi_corr_mean / len(roi_list)
print()
print("ROI CORR MEAN:", roi_corr_mean)
print()    
print()

print("Mean Abs Error: EXPERT1 vs Model (SUBSET")
roi_mae_mean = 0
for k,v in abs_diff_dict_e1_m_subset.items():
    print(k, v)
    if k in roi_list:
        roi_mae_mean += v[0]
roi_mae_mean = roi_mae_mean / len(roi_list)
print()
print("ROI MAE MEAN:", roi_mae_mean)
print()    
print()   
print()
print("Coefficient of Variation: EXPERT1 vs EXPERT2")
for k,v in cv_e1_e2.items():
    print(k, v)
    
    
    
print()
print("Cohen's D: EXPERT1 vs EXPERT2")
for k,v in cohen_e1_e2.items():
    print(k, v)

    

### See how well they agree in terms of the average T2 change over time in each cartilage region of each image

In [ ]:
expert1_regions = list(expert1_pd.t2_region_json_path)
model_regions = list(predict_pd.t2_region_json_path)


source1_regions1 = [i for i in expert1_regions if i[-6]=='4']# and '9120358' not in i]
source1_regions2 = [i for i in expert1_regions if i[-6]=='8']#and '9120358' not in i]
source2_regions1 = [i for i in model_regions if i[-6]=='4']#and '9120358' not in i]
source2_regions2 = [i for i in model_regions if i[-6]=='8']#and '9120358' not in i]

if len(source1_regions1)==len(source1_regions2):
    catch = compare_region_changes(source1_regions1,
                                   source1_regions2,
                                   source2_regions1,
                                   source2_regions2, 
                                   results_path=None,
                                   correlation_method = 'spearman',
                                   bland_altman = True)  
    
    (change_correlation_dict, change_mean_abs_diff_dict, change_cv_dict, change_ttest_dict, change_dict, change_cohen) = catch
    
print("CORRELATION EXPERT 1 VS MODEL")
for k,v in change_correlation_dict.items():
    print(k, v)
    

print()
print("MEAN ABS DIFFERENCE EXPERT 1 VS MODEL")
for k,v in change_mean_abs_diff_dict.items():
    print(k, v)
print()

    
print()
print("MEAN CHANGE ACCORDING TO EXPERT 1")
for k,v in change_dict[1].items():
    print(k, np.mean(v))
    
    
print()
print("COHEN's D ACCORDING TO EXPERT 1")
for k,v in change_cohen.items():
    print(k, np.mean(v))


for k in change_dict[1].keys():
    plt.scatter(change_dict[1][k],change_dict[2][k]) 
    plt.title(k)
    plt.plot([0,0],[-5,5],'orange')
    plt.plot([-5,5],[0,0], 'orange')
    plt.plot([-1.73,-1.73],[0,-4])
    plt.plot([1.73, 1.73],[0,-4])
    plt.plot([-.61,-.61],[0,-4])
    plt.plot([.61, .61],[0,-4])
    plt.show()


### See how well they agree in terms of the cartilage lesions that are identified as developing over time

In [ ]:
plt.scatter(percentLesion_expert1, percentLesion_predict)
plt.title("Correlation:" + str(spearmanr(percentLesion_expert1, percentLesion_predict)))
plt.xlabel("Source 1: Fraction of cartilage affected")
plt.ylabel("Source 2: Fraction of cartilage affected")
plt.plot([.03,.15],[.03,.15])
plt.plot(np.unique(percentLesion_expert1), np.poly1d(np.polyfit(percentLesion_expert1, percentLesion_predict, 1))(np.unique(percentLesion_expert1)))
plt.legend(['unity','best fit','data points'])
plt.show()


bias_raw = np.mean(((np.array(percentLesion_expert1) - np.array(percentLesion_predict))))
StD_raw = np.std(np.abs((np.array(percentLesion_expert1) - np.array(percentLesion_predict))))
Mean_Abs_error = np.mean(np.abs((np.array(percentLesion_expert1) - np.array(percentLesion_predict))))
StD_Abs_error = np.std(np.abs((np.array(percentLesion_expert1) - np.array(percentLesion_predict))))


print()    
print("Raw Error:", bias_raw, "+/-", StD_raw)
print("Mean absolute error:", Mean_Abs_error)
print("StD absolute error:", StD_Abs_error)

bias_relative = np.mean(((np.array(percentLesion_expert1) - np.array(percentLesion_predict))/np.array(percentLesion_expert1)))
StD_relative = np.std(np.abs((np.array(percentLesion_expert1) - np.array(percentLesion_predict))/np.array(percentLesion_expert1)))
    
print()    
print("Relative Error:", bias_relative, "+/-", StD_relative)
 
print("Mean Lesion Coverage for Expert 1:", np.mean(percentLesion_expert1))
print("Mean Lesion Coverage for Model:", np.mean(percentLesion_predict))



In [ ]:
np.mean(np.abs((np.array(percentLesion_expert1) - np.array(percentLesion_predict))))



In [ ]:
a = np.load('../data/kevin_data/qmetric/t2maps/9120358_4.npy')
plt.imshow(a[19])

In [ ]:
with open('../data/kevin_data/qmetric/t2_projected/9909311_4.npy.pickle', 'rb') as handle:
    dict_a = pickle.load(handle)

with open('../data/kevin_data/qmetric/t2_projected/9909311_8.npy.pickle', 'rb') as handle:
    dict_b = pickle.load(handle)
    
a = make_projection_proportional(dict_a)
b = make_projection_proportional(dict_b)
a,b = align_projections(a,b)

In [ ]:
plt.imshow(a)
plt.show()

In [ ]:
plt.imshow(b)
plt.show()

In [ ]:
num_pix = np.sum(b!=0)
where_b = np.where(b)
where_a = np.where(a)

dmap = np.zeros_like(b)
for i in range(num_pix):
    t2_b = b[where_b[0][i], where_b[1][i]]
    
    distances_r = where_a[0] - where_b[0][i]
    distances_c = where_a[1] - where_b[1][i]
    distances = np.sqrt(distances_r**2 + distances_c**2)
    index = np.argmin(distances)
    t2_a = a[where_a[0][index], where_a[1][index]]
    
    change = t2_b - t2_a
    dmap[where_b[0][i], where_b[1][i]] = change
plt.imshow(dmap)

In [ ]:
from difference_map_utils import eroded_and_mask
mask = eroded_and_mask(dmap!=0,dmap!=0)
dmap = dmap*mask
plt.imshow(dmap)

In [ ]:
a= np.array([10,2,1,5])
i = np.argsort(a)
a[i]

In [ ]:
a=np.array([[10,1,5,3],[8,9,7,2],[1,2,3,4],[7,6,5,4]])
a

In [ ]:
a[[0,0,1],[2,3,3]]